In [1]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd


In [2]:
import neuralxpresso2 as nx
YT_LINK = "https://www.youtube.com/watch?v=cNqpJMYU9YI"
cb_palette = ['#CA3435', '#ff7f00', '#9370DB', '#40E0D0', '#1f77b4', '#9467bd', '#7f7f7f']


npx = nx.NeuralXpressoSession(yt_link=YT_LINK)
result = npx.run_analysis()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-03-28 08:28:19.328399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-28 08:28:19.328427: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-28 08:28:22.924050: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-28 08:28:23.042954: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 14ms/step


In [3]:
df_character = result['character_overview']
df_character

,person_ID,appearances,Angry,Disgust,Fear,Happy,Sad,Surprise,Neutral
0,1,24,0.061165,0.156956,0.185775,0.170192,0.030888,0.298332,0.096692
1,2,1,0.088927,0.000009,0.032039,0.003983,0.418394,0.448574,0.008074
2,3,2,0.085629,0.010206,0.164527,0.326802,0.064433,0.045604,0.302798


In [4]:
df_video = result['video_overview']
df_video

,frame,person_ID,emotion,probability
0,0,1,Angry,0.085881
1,1,1,Angry,0.090539
2,2,1,Angry,0.031928
3,3,1,Angry,0.053955
4,4,1,Angry,0.042333
...,...,...,...,...
184,22,1,Neutral,0.030466
185,23,1,Neutral,0.178083
186,23,2,Neutral,0.008074
187,24,3,Neutral,0.303137


In [5]:
def get_df_single_person(df_video, ID):
    df_single_person = df_video[df_video['person_ID'] == ID]
    df_single_person['moving_avg'] = df_single_person['probability'].rolling(window=10, center=True).mean()
    return df_single_person



In [6]:

def get_emotion_landscape(df_single_person):
    # Create the plot 
    fig = px.area(df_single_person, x="frame", y="moving_avg", color="emotion",
                color_discrete_sequence=cb_palette, hover_data={"text": df_single_person['emotion']})

    # Update the layout 
    fig.update_layout(
        title={
            'text': '<b>Probability of emotion per frame</b>',
            'font': {'size': 24, 'color': 'black'},
            'x': 0.5, #center header
            'y': 0.95 #center header
        },
        xaxis_title='Frames',
        yaxis_title='Probability',
        legend_title='Emotion',
        font=dict(family='Arial', size=14),
        margin=dict(l=50, r=50, t=100, b=50),
        plot_bgcolor='white',
        xaxis=dict(dtick=1),  #set the x-axis step 
        legend=dict(
            font=dict(size=10),
            xanchor='center',
            yanchor='top',
            x=0.5,
            y=-0.15
        )
    )

    # Update the legend with customizations
    fig.update_traces(
        hovertemplate='<br>'.join([
            'Emotion: %{fullData.name}',
            'Frame: %{x:.2f} ',
            'Probability: %{y:.2f}'
        ]),
        hoverlabel=dict(bgcolor='white', font_size=14),
        line=dict(width=2),
        showlegend=True,
        stackgroup='one',
        hoverinfo='all'
    )

    return fig


In [7]:
df_single_person = get_df_single_person(df_video, ID=1)

/var/folders/0p/tdp0bmnn6y9_vcys0nhkf7mw0000gn/T/ipykernel_7607/1087280200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_person['moving_avg'] = df_single_person['probability'].rolling(window=10, center=True).mean()


In [8]:
df_single_person

,frame,person_ID,emotion,probability,moving_avg
0,0,1,Angry,0.085881,NaN
1,1,1,Angry,0.090539,NaN
2,2,1,Angry,0.031928,NaN
3,3,1,Angry,0.053955,NaN
4,4,1,Angry,0.042333,NaN
...,...,...,...,...,...
181,19,1,Neutral,0.023096,0.089042
182,20,1,Neutral,0.088136,NaN
183,21,1,Neutral,0.019044,NaN
184,22,1,Neutral,0.030466,NaN


In [9]:
# Define a function that returns the emotion with the highest probability for a given row
def get_max_emotion(row):
    return row['emotion'][np.argmax(row['probability'])]

def get_df_radar(df_single_person):
    # Apply the function to each row of the DataFrame
    max_prob_rows = df_single_person.groupby('frame')['probability'].idxmax().reset_index()
    df_radar = df_single_person.loc[max_prob_rows['probability']]
    feeling_counts = df_radar.groupby('emotion')['frame'].nunique()

    return df_radar, feeling_counts




In [10]:
# Define the color palette
cb_palette = ['#40E0D0', '#ff7f00', '#9370DB', '#CA3435', '#1f77b4', '#9467bd', '#7f7f7f']

def get_radar_plot(feeling_counts):
    categories = ["angry",
                "disgust",
                "fear",
                "happy",
                "sad",
                "surprise",
                "neutral"]
    #values = [max_prob_df[max_prob_df['feeling'] == category]['frame'].nunique() for category in categories]

    # Define the categories and values
    #categories = feeling_counts.index
    values = feeling_counts.values

    # Create the radar chart
    trace = go.Scatterpolar(
        r = values,
        theta = categories,
        fill = 'toself',
        name = 'Prevalence of feelings'
    )

    # Set the layout for the chart
    layout = go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = False,
                range = [0, max(values)]
            )
        ),
        showlegend = False,
        title = 'Prevalence of feelings'
    )

    # Create the figure object and display the chart
    Fig = go.Figure(data=[trace], layout=layout)
    return trace, layout



In [11]:
df_single_person = get_df_single_person(df_video, ID=1)
df_radar, feeling_counts = get_df_radar(df_single_person)

# Create two traces and an image
image_array = result['portraits'][1]
fig_area = get_emotion_landscape(df_single_person)
trace_radar, layout = get_radar_plot(feeling_counts)



# Create a 3x1 grid of subplots with custom column widths
fig = make_subplots(rows=1, cols=3, column_widths=[0.2, 0.6, 0.2], specs=[[{'type': 'image'}, {'type': 'xy'}, {'type': 'polar'}]])

# Add trace1, trace2, and the image to their respective subplots
fig.add_trace(px.imshow(image_array).data[0], row=1, col=1)
for i in range(7):
    fig.add_trace(fig_area.data[i], row=1, col=2)
fig.add_trace(trace_radar, row=1, col=3)

#fig.add_trace

# Update layout

fig.update_layout(
    polar_radialaxis=layout['polar']['radialaxis'],
    title='Two Graphs and an Image Side by Side',
    height = 330,
    legend=dict(
        font=dict(size=10),
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.15,
        orientation='h')
    )




# Show the figure
fig.show()

/var/folders/0p/tdp0bmnn6y9_vcys0nhkf7mw0000gn/T/ipykernel_7607/1087280200.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_single_person['moving_avg'] = df_single_person['probability'].rolling(window=10, center=True).mean()


In [12]:
print(df_radar)
fig_radar = get_radar_plot(df_radar)

     frame  person_ID   emotion  probability  moving_avg
27       0          1   Disgust     0.611146    0.119654
136      1          1  Surprise     0.616187    0.208390
137      2          1  Surprise     0.332994    0.234460
84       3          1     Happy     0.340111    0.132254
58       4          1      Fear     0.311882    0.143322
140      5          1  Surprise     0.376593    0.343235
141      6          1  Surprise     0.361595    0.378503
142      7          1  Surprise     0.307259    0.319178
143      8          1  Surprise     0.393484    0.318958
144      9          1  Surprise     0.460170    0.340532
145     10          1  Surprise     0.445931    0.325444
92      11          1     Happy     0.820482    0.192888
147     12          1  Surprise     0.330799    0.278563
148     13          1  Surprise     0.411154    0.289340
176     14          1   Neutral     0.319877    0.095822
69      15          1      Fear     0.379490    0.204682
97      16          1     Happy

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()